In [ ]:
!pip install transformers datasets tf-models-official

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
from psutil import virtual_memory
from google.colab import files
import pandas as pd
import numpy as np
import io
import nltk
import re
import string
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from transformers import DataCollatorWithPadding
from transformers import TFAutoModel
from transformers import AutoTokenizer
from transformers import TFAutoModelForSequenceClassification
from datasets import Dataset
import tensorflow as tf
import tensorflow_models as tfm
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam, AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy, BinaryCrossentropy

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
uploaded = files.upload()

Saving multi_train_labels.csv to multi_train_labels.csv


In [ ]:
SEED = 2

In [ ]:
task2_df = pd.read_csv(io.BytesIO(uploaded['multi_train_labels.csv']))

In [ ]:
label_cols = ['G','L','B','T','O']
task2_df['labels'] = list(task2_df[label_cols].values)

In [ ]:
SEED = 2

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(list(task2_df['tuit']), task2_df['labels'],
                                                                    test_size=.15, shuffle=True, random_state=SEED)

In [ ]:
print(len(train_texts))
print(len(val_texts))

732
130


In [ ]:
checkpoint = "microsoft/mdeberta-v3-base" 
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
train_encodings['labels'] = train_labels
val_encodings['labels'] = val_labels

In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
batch_size = 32
num_epochs = 5

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = Dataset.from_dict(train_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

tf_validation_dataset = Dataset.from_dict(val_encodings).to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=batch_size,
)

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = AdamW(learning_rate=lr_scheduler)
loss=BinaryCrossentropy(from_logits=True)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5, problem_type="multi_label_classification",
                                                             attention_probs_dropout_prob=0.1,
                                                             hidden_dropout_prob=0.1,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDebertaV2ForSequenceClassification: ['mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight', 'deberta.embeddings.position_embeddings.weight', 'deberta.embeddings.word_embeddings._weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'deberta.embeddings.position_embeddings._weight']
- This IS expected if you are initializing TFDebertaV2ForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDebertaV2ForSequenceClassification f

In [ ]:
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=["accuracy"]
)

In [ ]:
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
    epochs=num_epochs
)

Epoch 1/5


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


23/23 [==============================] - 103s 1s/step - loss: 0.3770 - accuracy: 0.7869 - val_loss: 0.2623 - val_accuracy: 0.8462
Epoch 2/5
23/23 [==============================] - 27s 1s/step - loss: 0.2768 - accuracy: 0.8251 - val_loss: 0.2578 - val_accuracy: 0.8462
Epoch 3/5
23/23 [==============================] - 26s 1s/step - loss: 0.2715 - accuracy: 0.8251 - val_loss: 0.2484 - val_accuracy: 0.8462
Epoch 4/5
23/23 [==============================] - 26s 1s/step - loss: 0.2424 - accuracy: 0.8251 - val_loss: 0.1918 - val_accuracy: 0.8462
Epoch 5/5
23/23 [==============================] - 25s 1s/step - loss: 0.1999 - accuracy: 0.8265 - val_loss: 0.1807 - val_accuracy: 0.8462


In [ ]:
preds = model.predict(tf_validation_dataset)["logits"]
class_preds = [(r > 0).astype(np.int) for r in preds]

In [ ]:
print("mdeberta Accuracy Score:", round(accuracy_score(class_preds, list(val_encodings['labels'])), 2))

mdeberta Accuracy Score: 0.78


In [ ]:
print(classification_report(class_preds, list(val_encodings['labels'])))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96       114
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0

   micro avg       0.76      0.95      0.84       114
   macro avg       0.20      0.19      0.19       114
weighted avg       0.98      0.95      0.96       114
 samples avg       0.80      0.83      0.81       114



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("mdeberta F1 Score:", f1_score(class_preds, list(val_encodings['labels']), average=None)*100)

mdeberta F1 Score: [96.42857143  0.          0.          0.          0.        ]


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model.push_to_hub("cybersyn/mdeberta-homomex-track2", from_pt=True)

tf_model.h5:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tokenizer.push_to_hub("cybersyn/mdeberta-homomex-track2")

CommitInfo(commit_url='https://huggingface.co/cybersyn/mdeberta-homomex-track2/commit/35544769d20923eb1af405cad44c39fc58bb5662', commit_message='Upload tokenizer', commit_description='', oid='35544769d20923eb1af405cad44c39fc58bb5662', pr_url=None, pr_revision=None, pr_num=None)